In [1]:
import tensorflow as tf
import numpy
import data
import model

learning_rate = .001
grad_clip = 5.
batch_size = 64

Reading files: 08527   
Reading successful!


In [2]:
tf.reset_default_graph()
c = model.classifier(
    rnn_sizes=[128, 128, 128], 
    fc_sizes=[64, 32, 16],
    time_steps=1200,
    batch_size=batch_size)
feeder = data.batch_pool(batch_size)
batch = next(feeder)

# Loss, accuracy and optimizer

In [3]:
'''
    Name scope is good for graph definition for debugging in TensorBoard
'''
labels = tf.placeholder(tf.int64, [c.batch_size], 'labels')
global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')

with tf.name_scope('cross_entropy'):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=c.logits)
    with tf.name_scope('total'):
        loss = tf.reduce_mean(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(c.preds, 1), labels)
    with tf.name_scope('acc'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss)
    with tf.name_scope('gradient_clipping'):
        capped_gvs = [(tf.clip_by_value(grad, -grad_clip, grad_clip), var) 
                      for grad, var in gvs]
        
    opt = optimizer.apply_gradients(capped_gvs, global_step)

# Summaries

In [4]:
summaries = tf.summary.merge([
    [(tf.summary.histogram('gradient', grad), 
      tf.summary.histogram('variable', var)) for grad, var in gvs],
    tf.summary.scalar('loss', loss),
    tf.summary.scalar('accuracy', accuracy)
])

# Train test

In [5]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter('/tmp/tensorflow_logdir/', graph=sess.graph)
saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)
tf.global_variables_initializer().run()

In [6]:
for i, batch in enumerate(feeder):
    state = sess.run(c.zero_state)
    x_feed, label_feed, lens_feed = batch
    
    for idx in range(0, lens_feed.max(), c.time_steps):
        data_window = x_feed[:, idx:idx+c.time_steps]
        lens_window = lens_feed-idx
        
        feed_dict = {
            c.x:data_window,
            c.init_state:state,
            c.seq_len:lens_window,
            labels:label_feed
        }
        
        fetch_dict = {
            'opt' : opt,
            'state' : c.rnn_last_states,
            'step' : global_step,
            'loss' : loss,
            'acc' : accuracy,
            'sum' : summaries
        }
        
        fetched = sess.run(fetch_dict, feed_dict)
        state = fetched['state']
        if idx%(2 * c.time_steps) == 0:
            print('\r%d'%idx, end='', flush=True)
            writer.add_summary(summary=fetched['sum'], global_step=fetched['step'])
            
    if i%100 == 0: 
        saver.save(sess, c.get_checkpoint_path(), fetched['step'])
    print('\t', i, fetched['loss'], fetched['acc'])
        

16800	 0 0.896767 0.625
16800	 1 0.708403 0.703125
16800	 2 0.839523 0.65625
16800	 3 0.932737 0.578125
16800	 4 1.04351 0.546875
16800	 5 0.885754 0.59375
16800	 6 0.925986 0.625
16800	 7 0.817321 0.625
16800	 8 0.990868 0.625
16800	 9 0.911252 0.5625
16800	 10 0.919819 0.5625
16800	 11 0.806155 0.703125
16800	 12 0.919337 0.59375
16800	 13 1.12182 0.421875
16800	 14 0.959142 0.546875
16800	 15 1.01605 0.5625
16800	 16 0.907044 0.5625
16800	 17 0.866555 0.625
16800	 18 1.02129 0.484375
16800	 19 1.00296 0.578125
16800	 20 0.982482 0.546875
16800	 21 1.01608 0.46875
16800	 22 0.875286 0.640625
16800	 23 0.94555 0.5
16800	 24 0.923029 0.625
16800	 25 0.914011 0.59375
16800	 26 0.831556 0.5
16800	 27 0.900615 0.640625
16800	 28 0.935535 0.53125
16800	 29 0.834163 0.6875
16800	 30 0.769934 0.75
16800	 31 0.915303 0.609375
16800	 32 0.852863 0.640625
16800	 33 0.854967 0.671875
16800	 34 0.824289 0.640625
16800	 35 0.932153 0.609375
16800	 36 0.907618 0.609375
16800	 37 0.959064 0.53125
16

ValueError: Cannot feed value of shape (16, 1200, 1) for Tensor 'input/X:0', which has shape '(64, ?, 1)'